<a href="https://colab.research.google.com/github/psrajer/tfjs/blob/master/ML_Seg_v6c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.


In [ ]:
!pip install ultralytics
!pip install rasterio
!pip install geopandas
!pip install leafmap

from ultralytics import YOLO, checks, hub, SAM

checks()  # check library loads

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 42.5/235.7 GB disk)


In [ ]:

import os
import glob
import cv2
import geopandas as gpd
import numpy as np
import rasterio
from shapely.geometry import Polygon
from shapely.geometry import Point
import pandas as pd
import torch

from google.colab import output


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
output.enable_custom_widget_manager()

#******************************************************
#*********************************************************
# model name used

model_name = 'Water_Seg_v2.pt'
model_name_SAM = 'sam2.1_l.pt'
output_file = 'SHP_240731_Waterb'

# Inference Parameters
confidence_model = 0.05
iou_model = 0.70
device_type='cuda'
buffer_size_m = 3.0
accept_value = 0.07

sample_size = 1280

# Load the model (repository or custom trained)
#Google Drive Location for models, results and images
dir_model = '/content/Models/'
dir_path = '/content/Images/'
dir_result = '/content/Results/'
#******************************************************
#******************************************************

shapefile_path = dir_result + output_file + '.shp'


model_file = dir_model + model_name
model_file_SAM = dir_model + model_name_SAM

model = YOLO(model_file)
modelSAM = SAM(model_file_SAM)

#Capture training image info as a list
infer_images = []
results = []
Image_Origin_Emin = []
Image_Origin_Nmax = []
Image_Origin_Emax = []
Image_Origin_Nmin = []
Image_CRS = 2956 #default
count_images = 0
num_images = 0
i_path = []

for directory_path in glob.glob(dir_path):
  for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
    #print(img_path)
    i_path.append(img_path)

    src = rasterio.open(img_path)

    # transform rasterio plot to real world coords
    extent = [src.bounds[0], src.bounds[2], src.bounds[1], src.bounds[3]]
    Image_Origin_Emin.append(src.bounds[0])
    Image_Origin_Nmax.append(src.bounds[3])
    Image_Origin_Emax.append(src.bounds[2])
    Image_Origin_Nmin.append(src.bounds[1])


    # Handle the case where CRS is not available
    if src.crs is not None:
        Image_CRS = src.crs.to_epsg()
    else :
        Image_CRS = 2956


    img_read = cv2.imread(img_path, 1)
    # Resize images to a common shape if they have different sizes
    img_read = cv2.resize(img_read, (sample_size,sample_size))  # Replace (512, 512) with your desired common size
    infer_images.append(img_read)
    count_images +=1
    src.close() # free electrons - amd more critically memory from the image file

#Convert list to array for machine learning processing
infer_images = np.array(infer_images)




In [ ]:
# Run inference on image or images

total_masks = 0
image_mask = 0
image_count = 0
count_images = -1
geometries = []
class_labels = []
classNames = ['']
conf_labels = []
confNames = []
count_seg = 0
mask_geometries = []


def add_binary_column(gdf, column_name='Accept', condition=None):

    if condition is None:
        gdf[column_name] = 1  # Set all values to 1 if no condition is provided
    else:
        gdf[column_name] = condition.astype(int)  # Convert boolean to integer (0 or 1)

    return gdf

# Create an empty list to store the geometries and class labels
for image in infer_images:

  x_pixels = image.shape[1]
  y_pixels = image.shape[0]

# ML Inference on each image in list (YOLO)
  results = model.predict(
      image,
      conf=confidence_model,
      iou=iou_model,
      device=device_type,

      )

  count_images += 1
  print(f"Processed Image: {count_images+1} of {len(infer_images)}")
  print(f"Image: {i_path[count_images]}")

  # origin reset to real world coords
  N_Origin = Image_Origin_Nmax[count_images]
  E_Origin = Image_Origin_Emin[count_images]
  N_pixel_size = (Image_Origin_Nmax[count_images] - Image_Origin_Nmin[count_images]) / y_pixels
  E_pixel_size = (Image_Origin_Emax[count_images] - Image_Origin_Emin[count_images]) / x_pixels

  # Extract bounding boxes, classes, names, and confidences
  #boxesY = results[0].boxes.xyxy.tolist()
  classesY = results[0].boxes.cls.tolist()
  #namesY = results[0].names
  confidencesY = results[0].boxes.conf.tolist()

  # Iterate over results to extract information
  print(f"Number of YOLO Objects Detected: {len(results[0].boxes)}")

  for rY in results:
    count_seg = 0
    # Reshape the bounding boxes to (N, 4) format if necessary
    bboxes = rY.boxes.xyxy.cpu().numpy() # Get bounding boxes as a NumPy array

    if len(results[0].boxes) > 0 :
      # Pass the reshaped bounding boxes to modelSAM
      r = modelSAM(
          image,
          bboxes=bboxes,device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),

          ) #SAM Modelling

      print(f"Number of SAM Masks Detected: {len(r[0].masks)}")

      for box in r[0].boxes:

        mask_seg = r[0].masks[count_seg].xy
        polygon = mask_seg[0]
        mask_polygon =[]

        for x, y in polygon:
          #print('Mask XY:', x,y) # x,y is in image pixel coords
          transformed_x = Image_Origin_Emin[count_images] + x*E_pixel_size
          transformed_y = Image_Origin_Nmax[count_images] - y*N_pixel_size
          #print('Mask Trans XY:', transformed_x,transformed_y)
          mask_polygon.append((transformed_x, transformed_y))

        # Check if the mask_polygon has enough coordinates
        if len(mask_polygon) >= 4:
          mask_geometries.append(Polygon(mask_polygon))

          # Get the class index for the current segment
          class_index = int(classesY[count_seg])

          # Get the class label using the index
          class_label = rY[0].names[class_index]

          # append the class_label for this segment
          class_labels.append(class_label)


          # Get confidence scores
          #conf_label = box.conf[0].item()
          conf_label = confidencesY[count_seg]
          conf_labels.append(conf_label)


        count_seg += 1


# Create a GeoDataFrame from the list of geometries and class labels - pulled bounding boxes, only segments exported

segment_gdf = gpd.GeoDataFrame({'geometry': mask_geometries, 'class': class_labels, 'conf' : conf_labels, 'Source' : "Segment" , 'Image' : i_path[count_images]})

# Set the CRS for the combined GeoDataFrame

segment_gdf = segment_gdf.set_crs(epsg=Image_CRS)  #'epsg:2956'

# Calculate area and ensure it's of numeric type
segment_gdf["area"] = segment_gdf['geometry'].area.astype(float)

# Add a binary column based on a condition
condition = segment_gdf['conf'] > accept_value
segment_gdf = add_binary_column(segment_gdf, column_name = "Acc_Conf", condition=condition)

segment_gdf.to_file(shapefile_path, driver='ESRI Shapefile')



0: 1280x1280 18 waters, 25.7ms
Speed: 11.7ms preprocess, 25.7ms inference, 326.4ms postprocess per image at shape (1, 3, 1280, 1280)
Processed Image: 1 of 48
Image: /content/Images/240731_20cm_E_06.tif
Number of YOLO Objects Detected: 18

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 942.0ms
Speed: 10.6ms preprocess, 942.0ms inference, 28.7ms postprocess per image at shape (1, 3, 1024, 1024)
Number of SAM Masks Detected: 18

0: 1280x1280 50 waters, 14.1ms
Speed: 6.0ms preprocess, 14.1ms inference, 11.0ms postprocess per image at shape (1, 3, 1280, 1280)
Processed Image: 2 of 48
Image: /content/Images/240731_20cm_E_05.tif
Number of YOLO Objects Detected: 50

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1 25, 1 26, 1 27, 1 28, 1 29, 1 30, 1 31, 1 32, 1 33, 1 34, 1 35, 1 36, 1 37, 1 38, 1 39, 1 40, 1 41, 1 42, 1 43, 1

In [ ]:
#This cell will buffer and dissolve the initial geodataframe, save out shapefiles

shapefile_path_buffer = dir_result + output_file + '_buffer.shp'
# Apply the buffer, keeping a copy of the original geometries for dissolve
segment_gdf['o_geometry'] = segment_gdf['geometry'].copy() #copy the original geometry before we modify the geometry column
segment_gdf['b_geometry'] = segment_gdf.geometry.buffer(buffer_size_m)

# Overwrite original or create a new gdf to continue to work with buffered polygons
segment_gdf['geometry'] = segment_gdf['b_geometry']
# Remove the temporary buffered_geometry column before reassigning the geometry column
segment_gdf.drop(columns=['b_geometry'], inplace=True)
#Convert original_geometry to wkt format, so it is not treated as a geometry column.
segment_gdf['o_geometry'] = segment_gdf['o_geometry'].apply(lambda geom: geom.wkt)

segment_gdf.drop(columns=['o_geometry'], inplace=True)
# Save the buffered GeoDataFrame to a shapefile
segment_gdf.to_file(shapefile_path_buffer, driver='ESRI Shapefile')


In [ ]:
# Draw Map - optional - testing the locations

from google.colab import output
output.enable_custom_widget_manager()
import leafmap.foliumap as leafmap

# Define the style function
def type_style_function(feature):
    if feature['properties']['class'] == 'Haul_Truck':
        return {'fillColor': 'red', 'color': 'black'}
    elif feature['properties']['class'] == 'Equipment':
        return {'fillColor': 'green', 'color': 'black'}
    elif feature['properties']['class'] == 'Infrastructure':
        return {'fillColor': 'blue', 'color': 'black'}
    else:
        return {'fillColor': 'black', 'color': 'black'}


m = leafmap.Map(zoom=12)
m.add_basemap("ROADMAP")
m.add_basemap("TERRAIN")
m.add_xyz_service("qms.Google Satellite Hybrid")


m.add_gdf(segment_gdf, layer_name="Buffered Disturbances",zoom_control=True,info_mode="on_hover",
          zoom_to_layer = True,
          style_function=type_style_function,
          legend_kwds={
            "title": "Type Legend",
            "labels": ["Haul_Truck", "Equipment", "Infrastrucutre", "Other"],  # Match style_function categories
            "colors": ["red", "green", "blue","black"],  # Match style_function colors
          }

)

display(m)
